In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,StackingRegressor
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
import xgboost as xgb
from sklearn import linear_model
from sklearn import svm
import lightgbm as lgbm
from sklearn import pipeline
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from scipy.stats import boxcox
from scipy.special import inv_boxcox

% matplotlib inline

UsageError: Line magic function `%` not found.


In [77]:
df = pd.read_csv("../data/train.csv")
df_test=pd.read_csv("../data/test.csv")
y_full_score = pd.read_csv('../full-score.csv')
print(df.shape,df_test.shape)
df = df[df.GrLivArea < 4500]

(1460, 81) (1459, 80)


## Replace NaN values and Model may assume them to be numerical

In [305]:
def replace_nan(features):
    str_club = ["MSSubClass","OverallQual","OverallCond","YearBuilt","YearRemodAdd","GarageYrBlt",
           "MoSold","YrSold"]
    regress_cols = ["LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2",
                        "BsmtUnfSF","TotalBsmtSF","1stFlrSF","2ndFlrSF","LowQualFinSF","GarageArea","WoodDeckSF",
                        "OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal"]
    na_club = ["Alley","GarageType","FireplaceQu","GarageFinish","GarageQual","GarageCond","PoolQC",
              "Fence","MiscFeature"]

#     for cols in features.columns:
#         if cols in str_club:
#             features[cols] = features[cols].astype(str)
    # filling mean for continuos column
    for cols in features.columns:
        if cols in regress_cols:
            features[cols] = features[cols].fillna(features[cols].mean())

    # filling NA for these looking at data and description
    for cols in features.columns:
        if cols in na_club:
            features[cols] = features[cols].fillna("NA")

    temp = (regress_cols+na_club)
    # filling the most comman occuring value for the rest
    for cols in features.columns:
        if cols not in temp and features[cols].isna().sum() > 0:
            features[cols] = features[cols].fillna(features[cols].value_counts().idxmax())
    return features

In [38]:
def relation_with_price_graph(df):
    df=df.fillna(df.mean())
    cols = np.array(df.columns[1:-1])
    regress_cols = ["LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2",
                    "BsmtUnfSF","TotalBsmtSF","1stFlrSF","2ndFlrSF","LowQualFinSF","GarageArea","WoodDeckSF",
                    "OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal"]
    y=df.iloc[:,-1]
    temp_df=df.iloc[:,1:]
    for i in cols:
        plt.figure(figsize=(4,2), dpi = 130)
        if i not in regress_cols:
            sns.catplot(x=i, y="SalePrice", kind="box",data=temp_df)

In [39]:
def get_correlation_matrix(df):    
    sns.set(style="white")
    regress_cols = ["LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2",
                    "BsmtUnfSF","TotalBsmtSF","1stFlrSF","2ndFlrSF","LowQualFinSF","GarageArea","WoodDeckSF",
                    "OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal","SalePrice"]
    df_temp=df[regress_cols].copy()
    
    # Compute the correlation matrix
    corr = df_temp.corr()

    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=np.bool))

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(20, 10))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [281]:
def preprocess_data(X,test=False):
    regress_cols = ["LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2",
                    "BsmtUnfSF","TotalBsmtSF","1stFlrSF","2ndFlrSF","LowQualFinSF","GarageArea","WoodDeckSF",
                    "OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal","SalePrice",
                   "GrLivArea"]
#     regress_cols = ["Id","LotFrontage","LotArea","YearBuilt","YearRemodAdd","MasVnrArea","BsmtFinSF1","BsmtFinSF2",
#                 "BsmtUnfSF","TotalBsmtSF","1stFlrSF","2ndFlrSF","LowQualFinSF","GarageYrBlt","GarageArea","WoodDeckSF",
#                 "OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal","YrSold","SalePrice"]
#     drop_cols = ["BsmtFinSF2","LowQualFinSF","MiscVal","3SsnPorch"]
#     X=X.drop(drop_cols,axis=1)
#     X = X.drop(X['Id'])
    le = preprocessing.LabelEncoder()
    for cols in X:
        if (cols not in regress_cols):
            X[cols] = le.fit_transform(X[cols].astype(str))
#     X=X.fillna(X.mean())
    return X

## data before cleaning

In [10]:
df = replace_nan(df)
df_test = replace_nan(df_test)

In [ ]:
relation_with_price_graph(df)

In [ ]:
get_correlation_matrix(df)

## Get Dummy variables: They suck!

In [41]:
def dummy_vars(X):    
    linear_cols = ["Id","LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2",
                            "BsmtUnfSF","TotalBsmtSF","1stFlrSF","2ndFlrSF","LowQualFinSF","GarageArea","WoodDeckSF",
                            "OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal",
                   "YearBuilt","YearRemodAdd","GarageYrBlt","MoSold","YrSold","SalePrice","GrLivArea"]
    temp = [x for x in X.columns if x not in linear_cols]
    X = pd.get_dummies(X,columns=temp).reset_index(drop=True)
    return X

## Preprocessing

In [42]:
def get_df(df,df_test):
    y = df['SalePrice']
    train_features = df.drop(['SalePrice'], axis=1)
    test_features = df_test
    features = pd.concat([train_features, test_features]).reset_index(drop=True)
    features = features.drop(['Id'],axis=1)

    features = replace_nan(features)
    features = preprocess_data(features)
    # features = dummy_vars(features)
    X = features.iloc[:len(y), :]
    X_test = features.iloc[len(y):, :]
    return (X,y,X_test)

In [43]:
def save_to_file(model,test_data,lambda_):
    y = model.predict(test_data)
    y = inv_boxcox(y,lambda_)
    ans=pd.DataFrame(y)
    ans.index = np.arange(1461, len(ans)+1461)
    ans.to_csv('output.csv',header=["SalePrice"],index_label=["Id"])

## Tuning Paramerters of models

In [361]:
def tune_parameters_gbr(model,X,y,test_data,y_full_score,lambda_):
    parameters = {'learning_rate': sp_randFloat(),
                  'n_estimators' : sp_randInt(100, 2000),
                  'max_depth' : sp_randInt(2, 12),
                  'subsample': sp_randFloat()
                 }
    
    randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                               cv = 5, n_iter = 10, n_jobs=-1)
    randm.fit(X, y)
    err(randm,test_data,y_full_score,lambda_)
    return randm

def tune_parameters_lgbr(model,X,y,test_data,y_full_score,lambda_):
    parameters = {'learning_rate': sp_randFloat(),
                  'n_estimators' : sp_randInt(100, 5000),
                  'max_bin' : sp_randInt(100, 300),
                  'num_leaves' : sp_randInt(2, 30),
                  'bagging_fraction': sp_randFloat(),
                  'subsample'    : sp_randFloat(),
                  'colsample_bytree': sp_randFloat()
                 }
    
    randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                               cv = 5, n_iter = 100, n_jobs=-1)
    randm.fit(X, y)
    err(randm,test_data,y_full_score,lambda_)
    return randm

def tune_parameters_xgb(model,X,y,test_data,y_full_score,lambda_):
    parameters = {'learning_rate': sp_randFloat(),
                  'n_estimators' : sp_randInt(100, 5000),
                  'max_depth' : sp_randInt(2, 12),
                  'subsample': sp_randFloat(),
                  'colsample_bytree': sp_randFloat()
                 }
    
    randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                               cv = 5, n_iter = 100, n_jobs=-1)
    randm.fit(X, y)
    err(randm,test_data,y_full_score,lambda_)
    return randm

## Error with the Real result

In [411]:
def err(model,test_data,y_full_score,lambda_):
    y = model.predict(test_data)
#     y[np.where(y<=0)] = 1
    y_final = np.array(y_full_score.iloc[:,-1])
    y = inv_boxcox(y,lambda_)
    temp=(np.log(y)-np.log(y_final))
    temp=np.sqrt(np.mean(temp**2))
    print(temp)

## Optimized model before droping Columns

In [48]:
def create_optimized_models():
    gbr = GradientBoostingRegressor(
        learning_rate= 0.04822281322390631,
        max_depth= 4,
        n_estimators= 1784,
        subsample= 0.672836260394989
    )
    
    lgbr = lgbm.LGBMRegressor(
        bagging_fraction= 0.24259298294498244,
        colsample_bytree= 0.10555358953643856,
        learning_rate= 0.02019821484348372,
        max_bin= 242,
        n_estimators= 7018,
        num_leaves= 6,
        subsample= 0.1345545567884744
    )
    xgbr = xgb.XGBRegressor(
        colsample_bytree= 0.47024425419361326,
        learning_rate= 0.02240271545343664,
        max_depth= 4,
        n_estimators= 2170,
        subsample= 0.8283716383359659
    )
    return gbr,lgbr,xgbr

## optimized models after droping columns and fixing skew

In [484]:
def create_optimized_models_skew():
    gbr = GradientBoostingRegressor(
        learning_rate= 0.028418669171278887,
        max_depth= 7,
        n_estimators= 1407,
        subsample= 0.36659208944052435
    )
    
    lgbr = lgbm.LGBMRegressor(
        bagging_fraction= 0.7596323409518068,
        colsample_bytree= 0.2895341969083036,
        learning_rate= 0.026228791957937814,
        max_bin= 230,
        n_estimators= 1517,
        num_leaves= 13,
        subsample= 0.7519907337203593
    )
    xgbr = xgb.XGBRegressor(
        colsample_bytree= 0.11300308215161059,
        learning_rate= 0.017276460305797103,
        max_depth= 3,
        n_estimators= 4318,
        subsample= 0.8245317104774829
    )
    
    KRR = KernelRidge(kernel='polynomial',coef0=2.5,alpha = 0.45200876633573595, degree = 1)
    
    ENet = make_pipeline(RobustScaler(), ElasticNet(
    alpha= 0.0027644272078725107, 
    l1_ratio= 0.6305717058417957,
    random_state= 15))
    
    lasso = make_pipeline(RobustScaler(), Lasso(
    alpha= 0.0049556384660235375,
    random_state= 23))
    
    return gbr,lgbr,xgbr,lasso,ENet,KRR

In [258]:
X,y,test_data=get_df(df,df_test)
y_box, lambda_ = boxcox(y)
print(X.shape,y.shape,test_data.shape,y_box.shape,lambda_)
gbr, lgbr,xgbr = create_optimized_models()

(1458, 79) (1458,) (1459, 79) (1458,) -0.07712951475421644


/home/shad3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


## Without droping columns

In [259]:
for i in [gbr, lgbr,xgbr]:
    i.fit(X,y_box)
    err(i,test_data,y_full_score,lambda_)

0.3937913328966712
0.3914424822660915
0.45610610412475505


## here we drop column in pre-process

In [421]:
df_train=pd.read_csv("../data/train.csv")
df_test=pd.read_csv("../data/test.csv")
df_train = df_train[df_train.GrLivArea < 4500]

total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# print(missing_data.head(20))

df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_test = df_test.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)

# df_train = preprocess_data(df_train)
# df_test = preprocess_data(df_test)

y_n = df_train['SalePrice']
test_data = df_test.drop(['Id'],axis=1)
df_train = df_train.drop(['Id','SalePrice'],axis=1)
test_data = replace_nan(test_data)

features = pd.concat([df_train, test_data]).reset_index(drop=True)
features = dummy_vars(features)

X_n = features.iloc[:len(y_n), :]
test_data = features.iloc[len(y_n):, :]

# test_data = dummy_vars(test_data)
# df_train = dummy_vars(df_train)
y_n_box, lambda_ = boxcox(y_n)
gbr_new, lgbr_new,xgbr_new = create_optimized_models_skew()

## After droping columns

In [422]:
# y_n = df_train['SalePrice']
# # X_n = df_train.drop(['Id','SalePrice'],axis=1)
# y_n_box, lambda_ = boxcox(y_n)
for i in [gbr_new,lgbr_new,xgbr_new]:
    i.fit(X_n,y_n_box)
    err(i,test_data,y_full_score,lambda_)

0.13799633226841798
0.13142629126165878
0.1312013247062314


In [352]:
regressor = xgb.XGBRegressor()
xgb_dummy = tune_parameters_xgb(regressor,X_n,y_n_box,test_data,y_full_score,lambda_)
print("\n The best parameters across ALL searched params:\n",
          xgb_dummy.best_params_)

0.13741793193822807

 The best parameters across ALL searched params:
 {'colsample_bytree': 0.8794988040295653, 'learning_rate': 0.4479943613065446, 'max_depth': 2, 'n_estimators': 7131, 'subsample': 0.9877609228494757}


In [353]:
regressor = lgbm.LGBMRegressor(objective='regression')
lgbm_dummy = tune_parameters_lgbr(regressor,X_n,y_n_box,test_data,y_full_score,lambda_)
print("\n The best parameters across ALL searched params:\n",
          lgbm_dummy.best_params_)

0.1309787221468581

 The best parameters across ALL searched params:
 {'colsample_bytree': 0.8794988040295653, 'learning_rate': 0.4479943613065446, 'max_depth': 2, 'n_estimators': 7131, 'subsample': 0.9877609228494757}


In [362]:
regressor = GradientBoostingRegressor()
gbr_dummy = tune_parameters_gbr(regressor,X_n,y_n_box,test_data,y_full_score,lambda_)
print("\n The best parameters across ALL searched params:\n",
          gbr_dummy.best_params_)

0.16413726935597472

 The best parameters across ALL searched params:
 {'learning_rate': 0.21165708825976437, 'max_depth': 7, 'n_estimators': 540, 'subsample': 0.39210125553893194}


In [395]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge

In [405]:
def tune_lasso(model,X,y,test_data,y_full_score,lambda_):
    parameters = {'lasso__alpha': sp_randFloat(),
                  'lasso__random_state' : sp_randInt(1, 100)
                 }
    
    randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                               cv = 5, n_iter = 100, n_jobs=-1)
    randm.fit(X, y)
    err(randm,test_data,y_full_score,lambda_)
    return randm

def tune_enet(model,X,y,test_data,y_full_score,lambda_):
    parameters = {'elasticnet__alpha': sp_randFloat(),
                  'elasticnet__l1_ratio': sp_randFloat(),
                  'elasticnet__random_state' : sp_randInt(1, 100)
                 }
    
    randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                               cv = 5, n_iter = 100, n_jobs=-1)
    randm.fit(X, y)
    err(randm,test_data,y_full_score,lambda_)
    return randm

def tune_krr(model,X,y,test_data,y_full_score,lambda_):
    parameters = {'alpha': sp_randFloat(),
                  'degree' : sp_randInt(1, 100)
                 }
    
    randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                               cv = 5, n_iter = 100, n_jobs=-1)
    randm.fit(X, y)
    err(randm,test_data,y_full_score,lambda_)
    return randm

In [406]:
lasso = make_pipeline(RobustScaler(), Lasso())
ENet = make_pipeline(RobustScaler(), ElasticNet())
KRR = KernelRidge(kernel='polynomial',coef0=2.5)

In [423]:
lasso_tuned = tune_lasso(lasso,X_n,y_n_box,test_data,y_full_score,lambda_)
print("\n The best parameters across ALL searched params:\n",
          lasso_tuned.best_params_)

0.1849039793883354

 The best parameters across ALL searched params:
 {'lasso__alpha': 0.0049556384660235375, 'lasso__random_state': 23}


In [419]:
krr_tuned = tune_krr(KRR,X_n,y_n_box,test_data,y_full_score,lambda_)
print("\n The best parameters across ALL searched params:\n",
          krr_tuned.best_params_)

0.15653674014026692

 The best parameters across ALL searched params:
 {'alpha': 0.45200876633573595, 'degree': 1}


In [420]:
enet_tuned = tune_enet(ENet,X_n,y_n_box,test_data,y_full_score,lambda_)
print("\n The best parameters across ALL searched params:\n",
          enet_tuned.best_params_)

0.1694584481752954

 The best parameters across ALL searched params:
 {'elasticnet__alpha': 0.0027644272078725107, 'elasticnet__l1_ratio': 0.6305717058417957, 'elasticnet__random_state': 15}


In [480]:
gbr_final, lgbr_final, xgbr_final, lasso_final, enet_final, krr_final = create_optimized_models_skew()

In [481]:
for i in [gbr_final, lgbr_final, xgbr_final, lasso_final, enet_final, krr_final]:
    i.fit(X_n,y_n_box)

In [487]:
vr = ensemble.VotingRegressor([('gb', gbr_final), ('xgbr', krr_final)])

In [469]:
final_pred = (
    0.4*lgbr_final.predict(test_data)+
    0.3*xgbr_final.predict(test_data)+
    0.3*gbr_final.predict(test_data)
    )

In [489]:
final_pred = vr.predict(test_data)
y_final = np.array(y_full_score.iloc[:,-1])
final_pred = inv_boxcox(final_pred,lambda_)
temp=(np.log(final_pred)-np.log(y_final))
temp=np.sqrt(np.mean(temp**2))
print(temp)

0.1402497984251759


In [488]:
vr.fit(X_n,y_n_box)

VotingRegressor(estimators=[('gb',
                             GradientBoostingRegressor(learning_rate=0.028418669171278887,
                                                       max_depth=7,
                                                       n_estimators=1407,
                                                       subsample=0.36659208944052435)),
                            ('xgbr',
                             KernelRidge(alpha=0.45200876633573595, coef0=2.5,
                                         degree=1, kernel='polynomial'))])